<a href="https://colab.research.google.com/github/Arunsai65/FMML_ASSIGNMENTS/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-16-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

1)* In machine learning, the size of the validation set plays a crucial role in assessing the performance of a model during training and in preventing overfitting. The validation set is used to estimate how well a model generalizes to unseen data, and its size can have different effects on the accuracy and performance metrics of the model. Let's explore how changing the percentage of the validation set affects the model's accuracy and behavior:

Increasing the Percentage of Validation Set:

Pros:
As you allocate more data to the validation set, you get a better estimate of the model's generalization performance because the validation set is representative of a larger portion of your dataset.
It helps in detecting overfitting early since the model has less training data to memorize.
Cons:
You have less data available for training, which might lead to slower model convergence or less accurate model parameters.
If the validation set becomes too large, you might not have enough data for effective model training, and the model might underfit.
The accuracy on the validation set is likely to be a more reliable estimate of the model's generalization performance as you increase the size of the validation set. However, the training performance (accuracy on the training set) may decrease as more data is moved to the validation set.

Reducing the Percentage of Validation Set:

Pros:
You have more data available for training, which can lead to better model parameter estimation and potentially higher training performance.
The model may converge faster during training.
Cons:
A smaller validation set may lead to a less reliable estimate of the model's generalization performance, as it might not be representative of the entire dataset.
Overfitting may be less apparent, as the model has more data to memorize.
With a smaller validation set, the accuracy on the validation set may be a less reliable indicator of generalization performance. The model's training performance is likely to be higher due to having more training data.

In practice, the size of the validation set is often determined empirically through techniques like cross-validation. Cross-validation involves splitting the dataset into multiple folds and training/validating the model on different subsets to get a more robust estimate of its performance. This can help mitigate the impact of the validation set size on model assessment.

*2)* The size of the training and validation sets can significantly affect how well you can predict the accuracy on the test set using the validation set. In machine learning, this is often referred to as the reliability of the validation set as a proxy for test set performance. Here's how the sizes of these sets can impact this prediction:

Larger Training Set:

When you allocate a larger portion of your data to the training set, the model has more data to learn from. This typically leads to better model parameter estimation and can result in a model that is more representative of the underlying data distribution.
With a larger training set, the model is more likely to generalize well to unseen data, assuming it is not overfitting.
As a result, the validation set's performance (e.g., accuracy) tends to be a better predictor of the test set performance. If the model performs well on a large validation set, it's more likely to perform well on the test set.
Larger Validation Set:

A larger validation set can provide a more reliable estimate of the model's generalization performance because it's based on a larger sample of the data. This means that the validation set's performance is a better reflection of how the model is likely to perform on unseen data.
However, if you allocate too much data to the validation set, the training set becomes smaller, which can lead to slower convergence and potentially less accurate model parameters.
Despite a larger validation set, it's still essential to ensure that the training set is representative and provides sufficient data for the model to learn effectively.
Balanced Split:

A balanced split between the training and validation sets strikes a compromise between having enough data for training and a representative subset for validation.
This balance ensures that the model has a good chance of learning the underlying patterns in the data while still getting a reliable estimate of its generalization performance.
The performance on the validation set in a balanced split is a reasonably good indicator of the model's performance on the test set.
the sizes of the training and validation sets are interrelated, and their relative sizes can affect how well you can predict the accuracy on the test set using the validation set. A well-chosen balance that considers the complexity of your model, the size of your dataset, and the need for reliable estimates of generalization performance is crucial. Cross-validation techniques can also be useful to mitigate the impact of a specific data split on model assessment by repeatedly splitting the data into different training and validation sets.

*3)* The choice of the percentage to reserve for the validation set in a machine learning task is not one-size-fits-all and depends on several factors, including the size of your dataset, the complexity of your model, and the available computational resources. However, there are some common practices and guidelines to consider when determining the validation set size:

Rule of Thumb: 70/30 or 80/20 Split:

A common starting point is to use a 70/30 or 80/20 split for training and validation, respectively. This means you reserve 70% or 80% of your data for training and the remaining 30% or 20% for validation.
This split is a good starting point for moderate-sized datasets and models of moderate complexity. It allows the model to learn from a substantial portion of the data while still providing a reasonably large validation set for performance estimation.
Cross-Validation:

Cross-validation techniques, such as k-fold cross-validation, can be used to get a more robust estimate of model performance. Instead of a fixed validation set size, you partition your data into k subsets (folds), train and validate the model k times, and then average the performance scores.
Cross-validation can help mitigate the impact of a specific data split on model assessment and provides a more reliable estimate of generalization performance.
Data Size Considerations:

If you have a very large dataset, you can afford to allocate a smaller percentage for the validation set since you still have a substantial amount of data for training.
Conversely, if you have a very small dataset, you might need to allocate a larger percentage for validation to ensure you have enough data to assess performance reliably.
Model Complexity:

More complex models, which are prone to overfitting, may benefit from larger validation sets to detect overfitting early. In such cases, you might consider a larger percentage for validation.
Resource Constraints:

Consider your available computational resources. Smaller validation sets result in faster training times, which can be crucial when you have limited resources.
Plotting Learning Curves:

Plot learning curves that show the model's performance on both the training and validation sets as you increase the validation set size or vary the training/validation split ratio.
Observe how the performance changes with different split ratios and choose the one that provides a good balance between training and validation sizes while maintaining reliable performance estimates.
In practice, it's often a good idea to experiment with different validation set sizes and use cross-validation to determine the optimal split for your specific problem and dataset. Keep in mind that the goal is to achieve a balance where the model can learn effectively while still providing a reliable estimate of its generalization performance on unseen data.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


*ANSWER 1*

Yes, averaging validation accuracy across multiple splits can give more consistent results. Averaging the results across these iterations helps to reduce the impact of randomness in the data split and provides a more stable estimate of your model's performance.

*ANSWER 2*

Cross-validation, including averaging validation accuracy across multiple splits, provides a more accurate estimate of how well your model is likely to perform on unseen data compared to a single validation split.

*ANSWER 3*

The number of iterations (folds) in cross-validation can impact the estimate of your model's performance. In general, increasing the number of iterations can lead to a more reliable estimate, especially if your dataset is small.

*ANSWER 4*

Increasing the number of iterations in cross-validation can help when dealing with small datasets. It allows you to use your limited data more effectively by repeatedly splitting it into training and validation sets. However, there are limits to how much this can compensate for a very small dataset. If your dataset is extremely small, you might still face challenges related to overfitting or underfitting, and more data collection or data augmentation techniques may be necessary to improve model performance.